# Trabalho SIB

Trabalho realizado no âmbito da unidade curricular de Sistemas Inteligentes para a Bioinformática.

Grupo constituído por:

    - Angelina Eiras PG42861
    - Carina Gonçalves PG45466
    - Rute Castro PG45475
    - Sónia Carvalho PG42882

In [ ]:
#pip install git+https://github.com/jcapels/propythia.git@fix_dependencies

In [2]:
pip install propythia.zip --user

Processing c:\users\asus\onedrive - universidade do minho\carina\sib\propythia.zipNote: you may need to restart the kernel to use updated packages.
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'

  Created wheel for propythia: filename=propythia-2.0.0-py3-none-any.whl size=369524 sha256=a70077bf3af56aba5cb3dcd775143c4f8b354cba85e0f672bfce46a44bb1779e
  Stored in directory: C:\Users\Asus\AppData\Local\Temp\pip-ephem-wheel-cache-u0atq2g2\wheels\ce\76\29\a6b30aaccc44618c669c914d1552f0cac041aefcb6286f7806
Successfully built propythia
  Attempting uninstall: propythia
    Found existing installation: propythia 2.0.0
    Uninstalling propythia-2.0.0:
      Successfully uninstalled propythia-2.0.0


In [1]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install --upgrade pip

In [ ]:
pip install tensorflow==2.11.0

In [ ]:
pip install keras

In [ ]:
pip install umap-learn[plot]

In [ ]:
pip install bio-embeddings[all]

## Secção 1

**Exploração inicial e pré-processamento**

• Rever toda a documentação disponível sobre o conjunto de dados.

• Carregar o conjunto de dados e realizar uma análise exploratória do mesmo

• Realizar os passos necessários de preparação dos dados e pré-processamento, incluindo possivelmente a geração de atributos, a sua seleção, o tratamento de possíveis valores em falta, etc.

**Esta etapa deve corresponder à secção 1 do Notebook onde deverá:**

• descrever e caracterizar os dados atribuídos de acordo com a documentação/literatura existente;

• descrever sucintamente as características dos dados disponíveis a partir da análise exploratória inicial;

• descrever os passos de preparação dos dados e pré-processamento que efetuou, justificando as suas escolhas;

• incluir os gráficos exploratórios iniciais que ilustrem as principais características dos dados

In [1]:
import pandas as pd
import numpy as np
import re
import random
import requests as r
from Bio import SeqIO
from io import StringIO
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras
from keras.optimizers.schedules import *

from propythia import *
from propythia.protein_descriptores import ProteinDescritors

from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.feature_selection import SelectKBest, SelectFromModel
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn import model_selection
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import auc, roc_curve, matthews_corrcoef, f1_score, roc_auc_score, precision_recall_curve
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

In [ ]:
df = pd.read_csv("train.csv", delimiter=';')
df_test = pd.read_csv("test.csv")
df_update = pd.read_csv("train_updates_20220929.csv")
df

In [ ]:
df_update

In [ ]:
df_test

In [ ]:
#update dos numeros de seq_id no df inicial
for i in df_update['seq_id']:
    df[df['seq_id'] == i] = df_update[df_update['seq_id'] == i]

#nao precisamos dos links
df.drop(['data_source'], axis=1, inplace=True)
df_test.drop(['data_source'], axis=1, inplace=True)

df.dropna(axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)

df

In [ ]:
df.info(verbose=True)
df.head()

In [ ]:
from tensorflow import keras
from keras.optimizers.schedules import *


In [ ]:
from propythia import *

In [ ]:
# gerar descritores

descriptors_df = ProteinDescritors(dataset= df ,  col= 'protein_sequence')
descriptors_df.dataset

In [ ]:
descriptors_df.get_lenght(n_jobs=4)

In [ ]:
descriptors_df.get_aa_comp(n_jobs=4)

In [ ]:
descriptors_df

In [ ]:
## NAO CORRER - É O QUE VAMOS USAR
## COMO DEMORA MUITO, METI NUM EXCEL E É SÓ IMPORTAR ESSE PARA TERMOS O DATASET DIREITO

#all_descriptors = descriptors_df.get_all_physicochemical(ph=7, amide=False, n_jobs=4)
#all_descriptors
#all_descriptors.to_csv('descriptors.csv')

#### Importar dados fisico quimicos

In [ ]:
dataset = pd.read_csv("descriptors.csv", delimiter=';')
dataset

só correr uma vez !!!

In [ ]:
#drop das colunas que só têm valores = 0

data = dataset.drop(columns=['tot', 'hydrogen', 'single', 'double'], axis=1)

data

separar o x do y e normalizar os dados entre 0 e 1

In [ ]:
X = data.iloc[:, 3:-1]
y = data.iloc[:, -1]

# normalização dos dados entre zero e 1
scaler = MinMaxScaler(feature_range=(0, 1))
data_norm = scaler.fit_transform(X)
data_norm

# summarize transformed data
np.set_printoptions(precision = 3)
print(data_norm[0:5,:])

In [ ]:
data_norm

normalizar com standardscaler -- ver qual é melhor!!!

In [ ]:
scaler_st = StandardScaler().fit(X)
data_st = scaler_st.transform(X)

# summarize transformed data
np.set_printoptions(precision = 3)
print(data_st[0:5,:])

In [ ]:
y

In [ ]:
colname = X.columns
colname

##### PCA

PCA com dados iniciais (sem transformação)

In [ ]:
colors = ["green" if active else "red" for active in y]

principalComponents = PCA(n_components=2).fit_transform(X)
xs = principalComponents[:,0]
ys = principalComponents[:,1]

fig, axe = plt.subplots(figsize=(20,10))
for j in range(len(xs)): axe.plot(xs[j], ys[j], marker="o", color=colors[j], markeredgecolor="dimgrey")


PCA com transformação MinMaxScaler

In [ ]:
colors = ["green" if active else "red" for active in y]
# blue is 1 green is 0

principalComponents = PCA().fit_transform(data_norm)
xs = principalComponents[:,0]
ys = principalComponents[:,1]

fig, axe = plt.subplots(figsize=(20,10))
for j in range(len(xs)): axe.plot(xs[j], ys[j], marker="o", color=colors[j], markeredgecolor="dimgrey")

PCA com transformação StandardScaler

In [ ]:
colors = ["green" if active else "red" for active in y]


principalComponents = PCA().fit_transform(data_st)
xs = principalComponents[:,0]
ys = principalComponents[:,1]

fig, axe = plt.subplots(figsize=(20,10))
for j in range(len(xs)): axe.plot(xs[j], ys[j], marker="o", color=colors[j], markeredgecolor="dimgrey")


##### TSNE

TSNE com dados iniciais (sem transformação)

In [ ]:
embeded = TSNE(n_components=2,n_iter=1000).fit_transform(X)
fig, axe = plt.subplots(figsize=(20,10))
for j in range(len(embeded)): axe.plot(embeded[j,0],embeded[j,1],marker="o",color=colors[j],markeredgecolor="dimgrey")


TSNE com transformação MinMaxScaler

In [ ]:
embeded = TSNE(n_components=2,n_iter=1000).fit_transform(data_norm)
fig, axe = plt.subplots(figsize=(20,10))
for j in range(len(embeded)): axe.plot(embeded[j,0],embeded[j,1],marker="o",color=colors[j],markeredgecolor="dimgrey")

TSNE com transformação StandardScaler

In [ ]:
embeded = TSNE(n_components=2,n_iter=1000).fit_transform(data_st)
fig, axe = plt.subplots(figsize=(20,10))
for j in range(len(embeded)): axe.plot(embeded[j,0],embeded[j,1],marker="o",color=colors[j],markeredgecolor="dimgrey")

#### Feature selection

SelectPercentile com os dados entre 0 e 1 --- se quiseres StandardScaler -- mudar o 'data_norm' para 'data_st'

In [ ]:
# Select Percentile
feature_selection_percentile = SelectPercentile(chi2, percentile=25)
feature_selection_percentile.fit(data_norm, y)

selected_features_percentile = X.columns[feature_selection_percentile.get_support()]
selected_features_percentile.shape

In [ ]:
method_percentile = selected_features_percentile

lista_percentile = []
for i in method_percentile:
    lista_percentile.append(i)
lista_percentile

SelectKBest com os dados entre 0 e 1 --- se quiseres StandardScaler -- mudar o 'data_norm' para 'data_st'

In [ ]:
# SelectKBest
feature_selection_kbest = SelectKBest(chi2, k=25)
feature_selection_kbest.fit(data_norm, y)

selected_features_kbest = X.columns[feature_selection_kbest.get_support()]
selected_features_kbest.shape

In [ ]:
method_kbest = selected_features_kbest
lista_kbest = []
for i in method_kbest:
    lista_kbest.append(i)
lista_kbest

Variance Threshold com os dados entre 0 e 1 — se quiseres StandardScaler – mudar o ‘data_norm’ para ‘data_st’

In [ ]:
# Variance Threshold
feature_selection_variance = VarianceThreshold()
feature_selection_variance.fit(data_norm, y)

selected_features_variance = X.columns[feature_selection_variance.get_support()]
selected_features_variance.shape


In [ ]:
method_variance = selected_features_variance
lista_variance = []
for i in method_variance:
    lista_variance.append(i)
lista_variance

Select From Model com os dados entre 0 e 1 — se quiseres StandardScaler – mudar o ‘data_norm’ para ‘data_st’

In [ ]:
# Select From Model

feature_selection_from_model = SelectFromModel(estimator=LogisticRegression())
feature_selection_from_model.fit(data_norm, y)

selected_features_from_model = X.columns[feature_selection_from_model.get_support()]
selected_features_from_model.shape

In [ ]:
method_from_model = selected_features_from_model

lista_from_model = []
for i in method_from_model:
    lista_from_model.append(i)
lista_from_model

#### Fazer um consenso entre os métodos para verificar qual será o melhor valor de features

In [ ]:
lista_from_model # ver valor da lista
lista_variance   # ver valor da lista
lista_percentile # ver valor da lista
lista_kbest # ver valor da lista

In [ ]:
def compare():
    z = [ x for x in lista_variance #variance pq da um valor muito elevado
          if x in lista_percentile or x in lista_kbest or x in lista_from_model]
    print(z)

compare()

In [ ]:
z_todos = [ x for x in lista_variance
          if x in lista_percentile or x in lista_kbest or x in lista_from_model]
print(z_todos)
len(z_todos)

In [ ]:
z_reduzido = [ x for x in lista_variance
          if x in lista_percentile and x in lista_kbest and x in lista_from_model]
print(z_reduzido)
len(z_reduzido)

selecionar no dataframe as features que deram no consenso

In [ ]:
#dataset com menos

data_reduz_features = df[z_reduzido]
data_reduz_features

In [ ]:
# dataset com mais do consenso

data_all_features = df[z_todos]
data_all_features

##### PCA

In [ ]:

colors = ["green" if active else "red" for active in y]
x = len(data_all_features)

pca = PCA(n_components = x).fit_transform(data_all_features)
xs = principalComponents[:,0]
ys = principalComponents[:,1]

fig, axe = plt.subplots(figsize=(20,10))
for j in range(len(xs)): axe.plot(xs[j], ys[j], marker="o", color=colors[j], markeredgecolor="dimgrey")

meti a data_norm que é entre 0 e 1, se for a outra tem de se mudar para data_st

In [ ]:
pca.generate_dotplot(data_norm, y)

##### TSNE

In [ ]:
embeded = TSNE(n_components=2,n_iter=1000).fit(data_all_features)
fig, axe = plt.subplots(figsize=(20,10))
for j in range(len(embeded)): axe.plot(embeded[j,0],embeded[j,1],marker="o",color=colors[j],markeredgecolor="dimgrey")

Correlação entre features

In [ ]:
X.corr()

In [ ]:
sns.heatmap(X.corr())

#### Machine Learning

In [ ]:
# dividir os dados
X_train, X_test, y_train, y_test = train_test_split(data_reduz_features, y, test_size = 0.25, random_state = 500)


print('Train: ', X_train.shape, y_train.shape)
print('Test: ', X_test.shape, y_test.shape)

print('Train: ', Counter(x for x in y_train))
print('Test: ', Counter(x for x in y_test ))

Regressão e curva roc

In [ ]:
model = LogisticRegression()
model.fit(X_train,y_train)

In [ ]:
def score_test_set(model, X_test, y_test):
    # score test set
    y_pred = model.predict(X_test)
    try:
        y_prob = model.predict_proba(X_test)
    except:
        y_prob = None
    print('accuracy', accuracy_score(y_test, y_pred))
    print('MCC', matthews_corrcoef(y_test, y_pred))
    print('f1 score', f1_score(y_test, y_pred))
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    print('fdr', float(fp) / (tp + fp))
    print('sn', float(tp) / (tp + fn))
    print('sp', float(tn) / (tn + fp))

    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))


In [ ]:
score_test_set(model, X_test, y_test)
# ROC_AUC
# prediction = model.predict_proba(X_test)

preds = model.predict_proba(X_test)[:, 1]
fpr, tpr, threshold = roc_curve(y_test, preds)
roc_auc = auc(fpr, tpr)

plt.title('ROC Curve')
plt.plot(fpr, tpr, 'g', label='AUC = %0.3f' % roc_auc)
plt.legend(loc='lower right')
plt.plot([0, 1], [0, 1], 'r--')

plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

print('ROC AUC score:', round(roc_auc, 4))

Comparar os modelos com e sem pesos

In [ ]:
#Test and compare multiple models

models = [('LR', LogisticRegression()),
          ('LR-W', LogisticRegression(class_weight={0: 1, 1: 8})),
          ('RF', RandomForestClassifier()),
          ('RF-W', RandomForestClassifier(class_weight={0: 1, 1: 8})),
          ('KNN', KNeighborsClassifier()),
          ('SVM', SVC()),
          ('SVM-W', SVC(class_weight={0: 1, 1: 8})),
          ('DT', DecisionTreeClassifier()),
          ('DT-W', DecisionTreeClassifier(class_weight={0: 1, 1: 8}))]


names = []
results = []
for name, model in models:
    kfold = model_selection.KFold(n_splits=10)
    cross_val = model_selection.cross_val_score(model, X_train, y_train, cv=kfold, scoring='f1')
    results.append(cross_val)
    names.append(name)
    msg = "%s: %f SD:%f" % (name, cross_val.mean(), cross_val.std())
    print(msg)


plot dos resultados -- all models

In [ ]:
# Plot the results
fig = plt.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

#### SVM

In [ ]:
param_grid = {'C': [0.01,0.1,1],
#               'C': [0.1, 1, 10]
              'kernel': ['linear']}

#hyperparameter optimization algorithm
grid = GridSearchCV(SVC(), param_grid, cv=3, refit = True, verbose = 3)
# fitting the model for grid search
grid.fit(X_train, y_train)
print(grid.best_params_)

In [ ]:
#SCORE THE TEST SET
score_test_set(grid, X_test, y_test)

feature importance

In [ ]:
pd.Series(abs(grid.best_estimator_.coef_[0]), index=colname).nlargest(20).plot(kind='barh')

#### Regressão logistica sem peso com RandomizedSearchCV

In [ ]:
logistic = LogisticRegression(solver='saga', tol=1e-2, max_iter=200,random_state=0)

distributions = dict(C=uniform(loc=0, scale=4), penalty=['l2', 'l1'])

clf = RandomizedSearchCV(logistic, distributions, random_state=0)
clf
search = clf.fit(X_train, y_train)
search.best_params_

score_test_set(search, X_test, y_test)

pd.Series(abs(search.best_estimator_.coef_[0]), index=colname).nlargest(20).plot(kind='barh')

#### RandomForest sem pesos com RandomizedSearchCV

In [ ]:
param_grid = {'n_estimators': [10, 100, 200]}

#hyperparameter optimization algorithm
grid = RandomizedSearchCV(RandomForestClassifier(), param_grid, cv=3, refit = True, verbose = 3)
# fitting the model for grid search
grid.fit(X_train, y_train)
print(grid.best_params_)
#SCORE THE TEST SET
score_test_set(grid, X_test, y_test)

# feature importance for RF
pd.Series(abs(grid.best_estimator_.feature_importances_), index=colname).nlargest(20).plot(kind='barh')

## Notas importantes:

- fazer o mesmo para o dataset de teste
- verificar mais coisas para a parte de ML
- decidir que metodo vamos usar para normalizar os dados (MinMaxScaler ou StandardScaler)
- verificar o y que me dá erro quando o tento utilizar (não sei pq???)
- duvidas = como analisar os dados de tm aka y?
- meter como problema binario?? (ex: menor que 50 é 0, maior que 50 é 1) -- não sei se é eficaz ou se faz sentido

